In [231]:
# Importacion de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline

plt.style.use('default') # para graficos matplotlib
plt.rcParams['figure.figsize'] = (10, 6)

sns.set(style="whitegrid") # grid seaborn

pd.options.display.float_format = '{:20,.3f}'.format # notacion output

# Acceso a set de datos

In [232]:
path = "/home/seba/Escritorio/Datos/TP1/data/"
df_props_full = pd.read_csv(path + "train.csv")

# Conversión de tipos

In [233]:
df_props_full['fecha'] = pd.to_datetime(df_props_full['fecha'])

In [234]:
# Convierto todos los valores 1/0 a uint8
df_props_full['gimnasio'] = df_props_full['gimnasio'].astype('uint8')
df_props_full['usosmultiples'] = df_props_full['usosmultiples'].astype('uint8')
df_props_full['piscina'] = df_props_full['piscina'].astype('uint8')
df_props_full['escuelascercanas'] = df_props_full['escuelascercanas'].astype('uint8')
df_props_full['centroscomercialescercanos'] = df_props_full['centroscomercialescercanos'].astype('uint8')

In [235]:
# Convierto los representables en uint8. Utilizo el tipo de pandas UInt8Dtype para evitar conflicto con NaN
df_props_full['antiguedad'] = df_props_full['antiguedad'].astype(pd.UInt8Dtype())
df_props_full['habitaciones'] = df_props_full['habitaciones'].astype(pd.UInt8Dtype())
df_props_full['garages'] = df_props_full['garages'].astype(pd.UInt8Dtype())
df_props_full['banos'] = df_props_full['banos'].astype(pd.UInt8Dtype())

In [236]:
# Convierto los representables en uint16. Utilizo el tipo de pandas UInt16Dtype para evitar conflicto con NaN
df_props_full['metroscubiertos'] = df_props_full['metroscubiertos'].astype(pd.UInt16Dtype())
df_props_full['metrostotales'] = df_props_full['metrostotales'].astype(pd.UInt16Dtype())

In [237]:
# Convierto los representables en uint32. Utilizo el tipo de pandas UInt32Dtype para evitar conflicto con NaN
df_props_full['id'] = df_props_full['id'].astype(pd.UInt32Dtype())
df_props_full['idzona'] = df_props_full['idzona'].astype(pd.UInt32Dtype())
df_props_full['precio'] = df_props_full['precio'].astype(pd.UInt32Dtype())

# Analisis Fechas Publicaciones

In [238]:
df_props_full['year'] = df_props_full['fecha'].dt.year
df_props_full['month'] = df_props_full['fecha'].dt.month

<span style="color:green">first_fortnight va a tomar el valor 1 si es la primer quincena del mes, 0 si la segunda

In [239]:
df_props_full['first_fortnight'] = df_props_full['fecha'].apply(lambda fecha: 1 if fecha.day < 15 else 0)

In [240]:
df_props_full.groupby('first_fortnight').agg('size').to_frame()

,0
first_fortnight,
0,125186
1,114814


# Conversion MEX a USD

Utilizo un csv con información sobre el valor del dolar en México día a día a partir del 12/12/2011 hasta el 31/01/2017. La conversión del Peso Mexicano a Dolares se hace con el valor del dolar en la fecha de la publicación.
Link Datos: https://mx.investing.com/currencies/usd-mxn-converter#historical_data_converter

In [402]:
df_dollar = pd.read_csv(path + 'dollar.csv')
print(df_dollar.dtypes)
print('\n')
print(df_dollar.shape)
df_dollar.head(2)

Fecha       object
Cierre      object
Apertura    object
Máximo      object
Mínimo      object
% var.      object
dtype: object


(1344, 6)


,Fecha,Cierre,Apertura,Máximo,Mínimo,% var.
0,12.12.2011,0.07240,0.07350,0.07360,0.07200,-1.63
1,13.12.2011,0.07210,0.07240,0.07280,0.07190,-0.41


In [403]:
# Con describe identifico si hay valores nulos
df_dollar.describe()

,Fecha,Cierre,Apertura,Máximo,Mínimo,% var.
count,1342,1343,1343,1343,1343,1343
unique,1342,312,316,316,301,280
top,11.12.2015,0.07680,0.07640,0.07670,0.07660,0.00
freq,1,23,25,24,25,117


In [404]:
# Muestro las lineas con valores nulos
df_dollar[df_dollar.isna().any(axis=1)]

,Fecha,Cierre,Apertura,Máximo,Mínimo,% var.
1342,NaN,NaN,NaN,NaN,NaN,NaN
1343,NaN,Máximo:0.08380,Mínimo:0.04540,Diferencia:0.03840,Promedio:0.06895,% var.:-34.78261


In [405]:
# Analiso los últimos registros
df_dollar.loc[1340:1343]

,Fecha,Cierre,Apertura,Máximo,Mínimo,% var.
1340,30.01.2017,0.04810,0.04790,0.04840,0.04780,0.63
1341,31.01.2017,0.04800,0.04810,0.04840,0.04780,-0.21
1342,NaN,NaN,NaN,NaN,NaN,NaN
1343,NaN,Máximo:0.08380,Mínimo:0.04540,Diferencia:0.03840,Promedio:0.06895,% var.:-34.78261


Elimino las últimas dos filas, donde la anteúltima funciona como separador y la última es un analisis global de los datos

In [406]:
df_dollar = df_dollar.dropna()
df_dollar.describe()

,Fecha,Cierre,Apertura,Máximo,Mínimo,% var.
count,1342,1342,1342,1342,1342,1342
unique,1342,311,315,315,300,279
top,11.12.2015,0.07680,0.07640,0.07670,0.07660,0.00
freq,1,23,25,24,25,117


In [407]:
df_dollar['Cierre'] = pd.to_numeric(df_dollar['Cierre'])
df_dollar['Cierre'] = df_dollar['Cierre'].round(3)
df_dollar['Fecha'] = pd.to_datetime(df_dollar['Fecha'], format='%d.%m.%Y')
df_dollar = df_dollar.set_index('Fecha')
df_dollar = df_dollar.loc[:, 'Cierre'].to_frame()

In [408]:
# Rango de fechas
print(df_dollar.index.min())
print(df_dollar.index.max())

2011-12-12 00:00:00
2017-01-31 00:00:00


In [409]:
# Agrego fechas faltantes (Sabados y Domingos) con valor 0
idx = pd.date_range(start='2011-12-12', end='2017-01-31')
df_dollar = df_dollar.reindex(idx, fill_value=0)

In [413]:
df_dollar.head(8)

,Cierre
2011-12-12,0.072
2011-12-13,0.072
2011-12-14,0.072
2011-12-15,0.072
2011-12-16,0.072
2011-12-17,0.000
2011-12-18,0.000
2011-12-19,0.072


In [414]:
# Cuando se trata de una fecha que corresponde a un Sabado o Domingo no se tiene infromación sobre Cierre
# Le asigno el valor correspondiente al Viernes previo
for i in range(0, len(df_dollar)):
    if (df_dollar.iloc[i]['Cierre'] == 0):
        df_dollar.iloc[i]['Cierre'] = df_dollar.iloc[i-1]['Cierre']

In [416]:
df_dollar.head(8)

,Cierre
2011-12-12,0.072
2011-12-13,0.072
2011-12-14,0.072
2011-12-15,0.072
2011-12-16,0.072
2011-12-17,0.072
2011-12-18,0.072
2011-12-19,0.072


In [446]:
price_dates = df_props_full.loc[:,['fecha','precio','year','month','first_fortnight']]
price_dates['fecha'] = price_dates['fecha'].apply(lambda x: x.replace(hour=0, minute=0, second=0)) # Seteo tiempo a 00:00:00 para join
price_dates = price_dates.set_index('fecha')
price_dates = price_dates.join(df_dollar, how='left')
price_dates = price_dates.reset_index()
price_dates.rename(columns = {'index' : 'fecha'}, inplace=True)
price_dates.describe()

,precio,year,month,first_fortnight,Cierre
count,"240,000.000","240,000.000","240,000.000","240,000.000","240,000.000"
mean,"2,530,838.346","2,014.675",7.441,0.478,0.064
std,"2,152,551.798",1.358,3.563,0.500,0.011
min,"310,000.000","2,012.000",1.000,0.000,0.048
25%,"952,772.500","2,014.000",4.000,0.000,0.054
50%,"1,850,000.000","2,015.000",8.000,0.000,0.063
75%,"3,390,000.000","2,016.000",11.000,1.000,0.076
max,"12,525,000.000","2,016.000",12.000,1.000,0.084


In [447]:
price_dates.rename(columns = {'Cierre' : 'MEX_to_USD', 'precio' : 'Precio_MEX'}, inplace=True)
price_dates.head()

,fecha,Precio_MEX,year,month,first_fortnight,MEX_to_USD
0,2012-01-01,950000,2012,1,1,0.072
1,2012-01-01,970000,2012,1,1,0.072
2,2012-01-01,6500000,2012,1,1,0.072
3,2012-01-01,3590000,2012,1,1,0.072
4,2012-01-01,1120000,2012,1,1,0.072


In [448]:
price_dates['Precio_USD'] = price_dates['Precio_MEX'] * price_dates['MEX_to_USD']
price_dates['Precio_USD'] = price_dates['Precio_USD'].astype(int)
price_dates.head()

,fecha,Precio_MEX,year,month,first_fortnight,MEX_to_USD,Precio_USD
0,2012-01-01,950000,2012,1,1,0.072,68400
1,2012-01-01,970000,2012,1,1,0.072,69840
2,2012-01-01,6500000,2012,1,1,0.072,467999
3,2012-01-01,3590000,2012,1,1,0.072,258479
4,2012-01-01,1120000,2012,1,1,0.072,80640
